# Machine Learning Language Translation

In [1]:
# In tensorflo 1.14  we have to import required libraries
'''
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np
'''

'\nfrom keras.models import Model\nfrom keras.layers import Input, LSTM, Dense\nimport numpy as np\n'

In [2]:
# In tensorflo 2.0  we have to import required libraries

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
import numpy as np

In [3]:
batch_size = 64   # Batch size for training.
epochs = 24   # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000 # Number of samples to train on.

In [4]:
# path to the data txt file on disk.

data_path = "C:/Users/Dell/Desktop/datasets/fra-eng.txt"

In [5]:
# vectorize the data we are reading the input and target data
# each and every sentence read all the unique characters
input_texts = []
target_texts = []
input_characters = set() # all english characters
target_characters = set()
with open(data_path, 'r', encoding = 'utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split('\t')
    # we use "tab" as the "start sequence" character.
    # for the targets, and "\n" as "end sequence" character.
    
    target_taxt = "t" + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [6]:
print(input_characters) #these are english characters
print(len(input_characters))

{'o', 'd', '$', 'M', 'U', 'H', '!', 'P', 'A', 'c', '0', 'N', '3', 'J', 't', 'W', 'T', 'O', '.', 'V', 'I', 'j', 'é', '1', '%', 'p', 'y', 'q', 'l', 'S', 'm', '9', '2', '?', 'D', 'L', 'h', "'", 's', ':', 'f', 'Y', 'G', 'e', 'a', 'C', '7', ',', '&', '8', 'i', 'R', 'B', ' ', 'w', 'r', '6', 'x', 'K', 'g', 'v', 'b', 'E', 'n', 'k', '5', 'F', 'z', '-', 'u', 'Q'}
71


In [7]:
print(target_characters)  #these are french characters
print(len(target_characters))

{'0', 't', '\u202f', 'T', 'V', '.', 'I', 'j', 'é', '2', '«', 'û', 'a', 'î', 'K', 'Ç', 'g', 'b', 'E', 'n', 'ù', '(', 'M', 'U', 'H', 'A', 'c', 'N', '3', 'J', 'è', '\u2009', 'ô', 'À', '%', 'y', 'S', 'l', 'm', 'L', 'h', 's', ':', 'f', 'Y', ',', 'R', 'ç', 'r', '5', 'F', '-', 'Q', 'o', 'd', 'O', 'ê', '1', 'q', 'É', '9', '»', 'C', '8', 'i', '’', ' ', '\xa0', 'x', 'ï', 'v', 'ë', 'k', '?', 'z', 'u', '!', ')', 'p', 'â', 'Ê', 'D', "'", 'G', 'e', '&', 'œ', 'B', 'P', 'à'}
90


In [8]:
print(input_texts)

['Go.', 'Hi.', 'Hi.', 'Run!', 'Run!', 'Who?', 'Wow!', 'Fire!', 'Help!', 'Jump.', 'Stop!', 'Stop!', 'Stop!', 'Wait!', 'Wait!', 'Go on.', 'Go on.', 'Go on.', 'Hello!', 'Hello!', 'I see.', 'I try.', 'I won!', 'I won!', 'I won.', 'Oh no!', 'Attack!', 'Attack!', 'Cheers!', 'Cheers!', 'Cheers!', 'Cheers!', 'Get up.', 'Go now.', 'Go now.', 'Go now.', 'Got it!', 'Got it!', 'Got it?', 'Got it?', 'Got it?', 'Hop in.', 'Hop in.', 'Hug me.', 'Hug me.', 'I fell.', 'I fell.', 'I know.', 'I left.', 'I left.', 'I lied.', 'I lost.', 'I paid.', "I'm 19.", "I'm OK.", "I'm OK.", 'Listen.', 'No way!', 'No way!', 'No way!', 'No way!', 'No way!', 'No way!', 'No way!', 'No way!', 'No way!', 'Really?', 'Really?', 'Really?', 'Thanks.', 'We try.', 'We won.', 'We won.', 'We won.', 'We won.', 'Ask Tom.', 'Awesome!', 'Be calm.', 'Be calm.', 'Be calm.', 'Be cool.', 'Be fair.', 'Be fair.', 'Be fair.', 'Be fair.', 'Be fair.', 'Be fair.', 'Be kind.', 'Be nice.', 'Be nice.', 'Be nice.', 'Be nice.', 'Be nice.', 'Be nice.

In [9]:
print(len(input_texts))

10000


In [10]:
target_texts

['Va !',
 'Salut !',
 'Salut.',
 'Cours\u202f!',
 'Courez\u202f!',
 'Qui ?',
 'Ça alors\u202f!',
 'Au feu !',
 "À l'aide\u202f!",
 'Saute.',
 'Ça suffit\u202f!',
 'Stop\u202f!',
 'Arrête-toi !',
 'Attends !',
 'Attendez !',
 'Poursuis.',
 'Continuez.',
 'Poursuivez.',
 'Bonjour !',
 'Salut !',
 'Je comprends.',
 "J'essaye.",
 "J'ai gagné !",
 "Je l'ai emporté !",
 'J’ai gagné.',
 'Oh non !',
 'Attaque !',
 'Attaquez !',
 'Santé !',
 'À votre santé !',
 'Merci !',
 'Tchin-tchin !',
 'Lève-toi.',
 'Va, maintenant.',
 'Allez-y maintenant.',
 'Vas-y maintenant.',
 "J'ai pigé !",
 'Compris !',
 'Pigé\u202f?',
 'Compris\u202f?',
 "T'as capté\u202f?",
 'Monte.',
 'Montez.',
 'Serre-moi dans tes bras !',
 'Serrez-moi dans vos bras !',
 'Je suis tombée.',
 'Je suis tombé.',
 'Je sais.',
 'Je suis parti.',
 'Je suis partie.',
 "J'ai menti.",
 "J'ai perdu.",
 'J’ai payé.',
 "J'ai 19 ans.",
 'Je vais bien.',
 'Ça va.',
 'Écoutez !',
 "C'est pas possible\u202f!",
 'Impossible\u202f!',
 'En aucun ca

In [11]:
print(len(target_texts))

10000


In [12]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [13]:
print("Number of samples:", len(input_texts))  # number of sentences
print("Number of unique input tokens:", num_encoder_tokens) # number of unique input characters
print("Number of unique output tokens:", num_decoder_tokens) # number of unique output characters
print("Number of Max sequence length for inputs:", max_encoder_seq_length) # maximum length of  input sentences 
print("Number of Max sequence length for outputs:", max_decoder_seq_length) #maximum length of output sentences

Number of samples: 10000
Number of unique input tokens: 71
Number of unique output tokens: 90
Number of Max sequence length for inputs: 16
Number of Max sequence length for outputs: 57


In [14]:
# It gives the index values for the characters based on the sorted list

input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

In [15]:
input_token_index

{' ': 0,
 '!': 1,
 '$': 2,
 '%': 3,
 '&': 4,
 "'": 5,
 ',': 6,
 '-': 7,
 '.': 8,
 '0': 9,
 '1': 10,
 '2': 11,
 '3': 12,
 '5': 13,
 '6': 14,
 '7': 15,
 '8': 16,
 '9': 17,
 ':': 18,
 '?': 19,
 'A': 20,
 'B': 21,
 'C': 22,
 'D': 23,
 'E': 24,
 'F': 25,
 'G': 26,
 'H': 27,
 'I': 28,
 'J': 29,
 'K': 30,
 'L': 31,
 'M': 32,
 'N': 33,
 'O': 34,
 'P': 35,
 'Q': 36,
 'R': 37,
 'S': 38,
 'T': 39,
 'U': 40,
 'V': 41,
 'W': 42,
 'Y': 43,
 'a': 44,
 'b': 45,
 'c': 46,
 'd': 47,
 'e': 48,
 'f': 49,
 'g': 50,
 'h': 51,
 'i': 52,
 'j': 53,
 'k': 54,
 'l': 55,
 'm': 56,
 'n': 57,
 'o': 58,
 'p': 59,
 'q': 60,
 'r': 61,
 's': 62,
 't': 63,
 'u': 64,
 'v': 65,
 'w': 66,
 'x': 67,
 'y': 68,
 'z': 69,
 'é': 70}

In [16]:
target_token_index

{' ': 0,
 '!': 1,
 '%': 2,
 '&': 3,
 "'": 4,
 '(': 5,
 ')': 6,
 ',': 7,
 '-': 8,
 '.': 9,
 '0': 10,
 '1': 11,
 '2': 12,
 '3': 13,
 '5': 14,
 '8': 15,
 '9': 16,
 ':': 17,
 '?': 18,
 'A': 19,
 'B': 20,
 'C': 21,
 'D': 22,
 'E': 23,
 'F': 24,
 'G': 25,
 'H': 26,
 'I': 27,
 'J': 28,
 'K': 29,
 'L': 30,
 'M': 31,
 'N': 32,
 'O': 33,
 'P': 34,
 'Q': 35,
 'R': 36,
 'S': 37,
 'T': 38,
 'U': 39,
 'V': 40,
 'Y': 41,
 'a': 42,
 'b': 43,
 'c': 44,
 'd': 45,
 'e': 46,
 'f': 47,
 'g': 48,
 'h': 49,
 'i': 50,
 'j': 51,
 'k': 52,
 'l': 53,
 'm': 54,
 'n': 55,
 'o': 56,
 'p': 57,
 'q': 58,
 'r': 59,
 's': 60,
 't': 61,
 'u': 62,
 'v': 63,
 'x': 64,
 'y': 65,
 'z': 66,
 '\xa0': 67,
 '«': 68,
 '»': 69,
 'À': 70,
 'Ç': 71,
 'É': 72,
 'Ê': 73,
 'à': 74,
 'â': 75,
 'ç': 76,
 'è': 77,
 'é': 78,
 'ê': 79,
 'ë': 80,
 'î': 81,
 'ï': 82,
 'ô': 83,
 'ù': 84,
 'û': 85,
 'œ': 86,
 '\u2009': 87,
 '’': 88,
 '\u202f': 89}

#It shows how we can do one hot representation using numpy

Initially for the encoder input data is "Zeros"  using "np.zeros" put all zero's values based on the 
dimensions i have given over here.

encoder_input_data is a 3D array of shape (num_pairs, max_english_sentence_length, num_english_characters) 
containing a one-hot vectorization of the English sentences.


similarly decoder_input_data and decoder_target_data also the same particular way to assign the dimensions


here decoder_input_data containing avectorization of French sentences.

In [17]:
encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype = 'float32')
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype = 'float32')
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype = 'float32')

In [18]:
encoder_input_data

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [19]:
decoder_input_data

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [20]:
decoder_target_data

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [21]:
print(len(encoder_input_data))
print(len(decoder_input_data))
print(len(decoder_target_data))

10000
10000
10000


In [22]:
# It is using to convert onehot representation
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
            encoder_input_data[i, t, input_token_index[char]] = 1
    encoder_input_data[i, t +1:, input_token_index[' ']] = 1
    for t, char in enumerate(target_text):
        #decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1
        if t> 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character
            decoder_target_data[i, t - 1, target_token_index[char]] = 1
        decoder_input_data[i, t + 1:, target_token_index[' ']] = 1
        decoder_target_data[i, t:, target_token_index[' ']] = 1

In [23]:
 encoder_input_data[2]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [24]:
encoder_input_data[2].shape

(16, 71)

In [25]:
print(decoder_input_data[2])
print(encoder_input_data[2].shape)

[[0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]
(16, 71)


In [26]:
print(decoder_target_data[2])
print(decoder_target_data[2].shape)

[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]
(57, 90)


In [27]:
# Define the input sequence and process it.
# Here "latent_dim" is just like timestep

encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state = True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# we discard encoder_outputs and onlu keep the states.
encoder_states = [state_h, state_c]

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [28]:
encoder_inputs

<tf.Tensor 'input_1:0' shape=(?, ?, 71) dtype=float32>

In [29]:
encoder

In [30]:
encoder_outputs

<tf.Tensor 'lstm/strided_slice_7:0' shape=(?, 256) dtype=float32>

In [31]:
state_h

<tf.Tensor 'lstm/while/Exit_2:0' shape=(?, 256) dtype=float32>

In [32]:
state_c

<tf.Tensor 'lstm/while/Exit_3:0' shape=(?, 256) dtype=float32>

In [33]:
encoder_states

[<tf.Tensor 'lstm/while/Exit_2:0' shape=(?, 256) dtype=float32>,
 <tf.Tensor 'lstm/while/Exit_3:0' shape=(?, 256) dtype=float32>]

In [34]:
# Set up decoder, using 'encoder_states' as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.

decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)

decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [35]:
decoder_inputs

<tf.Tensor 'input_2:0' shape=(?, ?, 90) dtype=float32>

In [36]:
decoder_lstm

In [37]:
decoder_outputs

<tf.Tensor 'dense/truediv:0' shape=(?, ?, 90) dtype=float32>

In [38]:
_

<tf.Tensor 'lstm_1/while/Exit_3:0' shape=(?, 256) dtype=float32>

In [39]:
_

<tf.Tensor 'lstm_1/while/Exit_3:0' shape=(?, 256) dtype=float32>

In [40]:
decoder_dense

In [41]:
decoder_outputs

<tf.Tensor 'dense/truediv:0' shape=(?, ?, 90) dtype=float32>

In [42]:
# Define the model that will turn
# 'encoder_input_data' & 'decoder_input_data' into 'decoder_target_data'

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data, 
          batch_size = batch_size, epochs = epochs, validation_split = 0.24)

Train on 7600 samples, validate on 2400 samples
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/24
7600/7600 [==============================] - 98s 13ms/sample - loss: 1.2958 - acc: 0.9916 - val_loss: 1.2893 - val_acc: 1.0000
Epoch 2/24
7600/7600 [==============================] - 96s 13ms/sample - loss: 1.1027 - acc: 0.9995 - val_loss: 1.2379 - val_acc: 1.0000
Epoch 3/24
7600/7600 [==============================] - 98s 13ms/sample - loss: 0.9946 - acc: 0.9962 - val_loss: 1.0919 - val_acc: 0.9774
Epoch 4/24
7600/7600 [==============================] - 97s 13ms/sample - loss: 0.9312 - acc: 0.9912 - val_loss: 1.0423 - val_acc: 0.9903
Epoch 5/24
7600/7600 [==============================] - 104s 14ms/sample - loss: 0.8904 - acc: 0.9868 - val_loss: 1.0239 - val_acc: 0.9719
Epoch 6/24
7600/7600 [==============================] - 99s 13ms/sample - loss: 0.8617 - acc: 0.9840 - val_loss: 0.9878 - val_acc: 0.9783
Epoch 7/24
7600/7600 [=======